In [1]:
import os
import numpy as np
import torch
import cv2
from torchvision.transforms import ToTensor

root_dir = ('')
img_files = []
imgs_list = []
for item in os.listdir(root_dir):
    img_files.append(item)
    imgs_list.append(cv2.imread(os.path.join(root_dir, item)))
imgs = []
for img in imgs_list:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = ToTensor()(img)
    #resize = 256
    #img = Resize((resize, resize))(img)
    imgs.append(img)

label_str = 'Y'
labels = []
for img in img_files:
    if label_str in img:
        label = 1
        labels.append(label)
    else:
        label = 0
        labels.append(label)
print(type(labels))
labels = np.asarray(labels)
labels = torch.from_numpy(labels)
labels = labels.type(torch.int64)
print(type(labels))
print('total images:', len(imgs))
print('total labels:', len(labels))

<class 'list'>
<class 'torch.Tensor'>
total images: 253
total labels: 253


In [2]:
import numpy as np

test_ratio = .1
val_ratio = .1
leng = len(imgs)
ind = np.arange(leng)
np.random.shuffle(ind)

test_split = int(test_ratio * leng)
val_split = int(val_ratio * leng) + test_split
test_ind = ind[:test_split] 
val_ind = ind[test_split:val_split]
train_ind = ind[val_split:]

x_train = [imgs[i] for i in train_ind]
y_train = [labels[i] for i in train_ind]

x_val = [imgs[i] for i in val_ind]
y_val = [labels[i] for i in val_ind]

x_test = [imgs[i] for i in test_ind]
y_test = [labels[i] for i in test_ind]

print('train samples:', len(x_train))
print('val samples:', len(x_val))
print('test samples:', len(x_test))
print(x_train[0].shape)

train samples: 203
val samples: 25
test samples: 25
(252, 200, 3)


In [3]:
import torch
from torchvision.transforms import ToPILImage, Resize, Compose, ToTensor, Grayscale, Normalize
torch.manual_seed(0)
resize = 256
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transforms = Compose([
    ToPILImage(),
    Resize((resize, resize)),
    ToTensor(),
    Normalize(mean, std)
])

In [4]:
from torch.utils.data import Dataset, DataLoader

class TumorDataset(Dataset):
    
    def __init__(self, imgs, labels, transforms):
        
        self.imgs = imgs
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        
        return len(self.imgs)
        
    def __getitem__(self, idx):

        if self.transforms:
            return self.transforms(self.imgs[idx]), self.labels[idx]
        else:
            return self.imgs[idx], self.labels[idx]
train_dataset = TumorDataset(x_train, y_train, transforms=transforms) 
val_dataset = TumorDataset(x_val, y_val, transforms=transforms)
test_dataset = TumorDataset(x_test, y_test, transforms=transforms)

train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=12, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=12, num_workers=0)

In [5]:
for i in range(len(train_dataset)):
    print(train_dataset[i][0].shape)
    print(train_dataset[i][1])

torch.Size([3, 256, 256])
tensor(0)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(0)
torch.Size([3, 256, 256])
tensor(0)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(0)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(0)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(0)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(0)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(0)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(0)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
tensor(1)
torch.Size([3, 256, 256])
te

In [10]:
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torchvision.models import resnet50
from torch.autograd import Variable
from sklearn.metrics import accuracy_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = resnet50(pretrained=True)
model.to(device)
loss_fn = CrossEntropyLoss()
opt = Adam(model.parameters(), lr=.0001)
epochs = 5
val_interval = 2
classes = ['Non-Tumor', 'Tumor']

# Function to save the model
def saveModel():
    path = "C:\\Users\\walsh\\Downloads\\Python_ML\\MLenvTorch\\Data\\Classification\\models\\Classifier_1.pth"
    torch.save(model.state_dict(), path)


def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    

    accuracy = (100 * accuracy / total)
    return(accuracy)

def train(epochs):
    
    best_accuracy = 0.0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    model.to(device)

    for epoch in range(epochs):
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
            
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            opt.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            opt.step()
            running_loss += loss.item()
            if i % 50 == 49:    
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 50))
                # zero the loss
                running_loss = 0.0
        accuracy = testAccuracy()
        print('For epoch', epoch+1,'accuracy: %d %%' % (accuracy))
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

In [11]:
train(epochs=epochs)

The model will be running on cpu device
For epoch 1 accuracy: 72 %
For epoch 2 accuracy: 60 %
For epoch 3 accuracy: 84 %
For epoch 4 accuracy: 88 %
For epoch 5 accuracy: 92 %


In [10]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import os
import torchvision
from torchvision.transforms import ToPILImage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_dir = ('C:\\Users\\walsh\\Downloads\\Python_ML\\MLenvTorch\\Data\\Classification\\models')
model = ResNet50(pretrained=True)
model.load_state_dict(torch.load(os.path.join(model_dir, 'Classifier_1.pth')))

def imageshow(img):
    img = img / 2 + 0.5 
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def testBatch(batch_size): 
    images, labels = next(iter(test_loader))

    for i in range(batch_size):
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        imageshow(torchvision.utils.make_grid(images[i]))
        print('Predicted: ', ''.join('%5s' % classes[predicted[i]]))
        print('Real labels: ', ' '.join('%5s' % classes[labels[i]])) 

In [11]:
testBatch(10)

NameError: name 'test_loader' is not defined